In [ ]:
import sys
sys.path.append("../../") 

In [ ]:
from epimodel.epiparam import EpidemiologicalParameters

In [ ]:
ep = EpidemiologicalParameters()

In [ ]:
import pandas as pd

import numpy as np

import seaborn as sns

In [ ]:
germany_linelist = pd.read_csv('../../data/raw_data_w_sources/de_linelist.csv')

In [ ]:
de_ll = germany_linelist.rename({'Meldedatum': 'confirmation date', 'IstErkrankungsbeginn': 'valid', 'Refdatum': 'onsetdate'}, axis=1)

In [ ]:
de_ll = de_ll[de_ll['valid']==1]

In [ ]:
dates_df = de_ll[['confirmation date', 'onsetdate']]

In [ ]:
dates_df['confirmation date'] = pd.to_datetime(dates_df['confirmation date'])
dates_df['onsetdate'] = pd.to_datetime(dates_df['onsetdate'])

In [ ]:
dates_df = dates_df[dates_df['onsetdate'] > '2020/08/01']

In [ ]:
days = (dates_df['confirmation date'] - dates_df['onsetdate']).to_numpy().astype('timedelta64[D]').astype(np.float32)

In [ ]:
days[days>64] = 0
days[days<0] = 0

In [ ]:
np.var(days)

In [ ]:
np.mean(days)

In [ ]:
infection_to_confirmation = {
    'mean': np.mean(days),
    'sd' :np.std(days),
    'dist': 'gamma'
}

In [ ]:
incubation = {
    'mean' :5.1,
    'sd': 5.1*0.86,
    'dist': 'gamma'
}

In [ ]:
s = ep.generate_dist_samples(infection_to_confirmation, int(1e9)) + ep.generate_dist_samples(incubation, int(1e9))

In [ ]:
germany_pmf = ep.discretise_samples(s, 64)

In [ ]:
austria_pmf = [8.5e-4, 5.76e-3, 1.96e-2, 4.34e-2, 6.99e-2, 9.16e-2, 1.03e-1, 1.05e-1, 9.86e-2, 8.8e-2, 7.53e-2, 6.23e-2, 
               5.04e-2, 4.01e-2, 3.18e-2, 2.49e-2, 1.9e-2, 1.47e-2, 1.12e-2, 8.63e-3, 6.54e-3, 5.13e-3, 3.85e-3, 2.9e-3,
               2.34e-3, 1.84e-3, 1.42e-3, 1.09e-3, 9e-4]

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
ep.generate_pmf_statistics_str(germany_pmf.flatten())

In [ ]:
ep.generate_pmf_statistics_str(np.array(austria_pmf))

In [ ]:
plt.plot(germany_pmf.T, label='Germany Linelist')
plt.plot(austria_pmf, label='Austria (from ICL website)')
plt.title('Infection to Confirmation')
plt.legend()
plt.xlim([0, 32])

In [ ]:
sns.histplot(days)